In [ ]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
!pip install accelerate -U
import datasets
import numpy as np
import transformers
from transformers import AutoTokenizer, BertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertModel
from transformers import AutoConfig
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
model = 'bert-base-cased'
dset = 'mteb/amazon_reviews_multi'

# Preprocessing

The dataset includes reviews from multiple languages so we only import the english ones. The dataset also includes alot of useless data for us, we only need the reviews and their ratings so lets process everything else out.

In [ ]:
dataset = datasets.load_dataset(dset, name='en'); # imports the dataset.
# check it works
print(dataset);

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [ ]:
engDataset = datasets.load_dataset(dset, name='en'); # imports the dataset.
# check it works
print(engDataset);

# FOR TESTING make the dataset smaller
# engDataset["train"] = engDataset["train"].select(range(100000))

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [ ]:
engDataset = engDataset.shuffle() # shuffle the dataset for safety.
engDataset = engDataset.remove_columns(['id', 'label_text']) # removes everything that we don't need

# Tokenization and padding

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model) # get the basic AutoTokenizer

In [ ]:
# tokenizes one example
def tokenize_example(example):
    split = example['text'].split('\n\n'); # splits the sentace and title.
    return tokenizer.encode_plus(split[0], split[1],
             truncation='only_second',
             add_special_tokens=True,
             return_attention_mask=True,
             return_overflowing_tokens=False,
             return_special_tokens_mask=False,
             max_length=512,
             pad_to_max_length=False)


In [ ]:
# map the whole dset
eng_tokenized = engDataset.map(tokenize_example)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
print(eng_tokenized['train'][1])
print(tokenizer.decode(eng_tokenized['train'][1]['input_ids']))

# looks good to me.

{'text': "Made in China dont last\n\nWorked great never broke in a pair of new car/truck speakers before but i decided to this time and guess what after break in they sounded amazing for about a week then the left one blew but i think they are fake kickers anyway they're made in china i didn't know Stillwater was in china thats wat i get for not reading the label on the back while online if thay are real (kicker) speakers then shame on kicker if not shame on me", 'label': 1, 'input_ids': [101, 10865, 1107, 1975, 1274, 1204, 1314, 102, 6955, 1174, 1632, 1309, 2795, 1107, 170, 3111, 1104, 1207, 1610, 120, 4202, 7417, 1196, 1133, 178, 1879, 1106, 1142, 1159, 1105, 3319, 1184, 1170, 2549, 1107, 1152, 4234, 6929, 1111, 1164, 170, 1989, 1173, 1103, 1286, 1141, 8390, 1133, 178, 1341, 1152, 1132, 8406, 27219, 1116, 4050, 1152, 112, 1231, 1189, 1107, 5144, 1161, 178, 1238, 112, 189, 1221, 4209, 4669, 1108, 1107, 5144, 1161, 1115, 1116, 20049, 1204, 178, 1243, 1111, 1136, 3455, 1103, 3107, 1113,

# Fine tuning the BERT model for our classification

In [ ]:
# config
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

class BertConfig(PretrainedConfig):
    def __init__(self, hidden_bert=768, hidden_ffnn=24, num_labels=5, vocab_size=28996):
        self.hidden_ffnn = hidden_ffnn
        self.hidden_bert = hidden_bert
        self.num_labels = num_labels
        self.vocab_size=vocab_size

# Create the bert class
class Bert(PreTrainedModel):
    config_class = BertConfig
    def __init__(self, config):
        super(Bert, self).__init__(config)
        # hidden size of BERT (always 768), hidden size of our classifier, and number of labels (in this case 5)
        H_in = config.hidden_bert
        H = config.hidden_ffnn
        labels = config.num_labels

        # bert is our preloaded distilbert
        self.bert = BertModel.from_pretrained(model)

        # basic one layer feed forward network that outputs the labels.
        self.classifier = nn.Sequential(
            nn.Linear(H_in, H), # bert in.
            nn.ReLU(), # ReLU
            #nn.Dropout(0.5), #dropout if needed.
            nn.Linear(H, labels) # to output labels
        )
    def forward(self, input_ids, attention_mask, labels=None):

        # run the BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token for classification
        last_hidden_state = outputs[0][:, 0, :]

        # Feed tha last hidden state into the classifier. This outputs the labels.
        logits = self.classifier(last_hidden_state)

        # if there is labels so training
        if labels is not None:
          # calculates the loss.
          loss = torch.nn.CrossEntropyLoss();
          return (loss(logits,labels),logits);
        else:
          # if no labels, just return the logits
          return (logits,);

In [ ]:
# configuration for BERT
config = BertConfig(hidden_bert=768, hidden_ffnn=24, num_labels=5)

In [ ]:
#calculates the accuracy
# from datasets import load_metric
# accuracy = load_metric('accuracy')
accuracy = evaluate.load('accuracy');
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels); # calc accuracy

In [ ]:
model = Bert(config)

In [ ]:
# Training params. We optimize these later.
training_args = TrainingArguments(
    output_dir='./bert_reviews',
    evaluation_strategy = 'steps',
    logging_strategy = 'steps',
    eval_steps = 1500,
    logging_steps = 1500,
    save_steps = 1500,
    learning_rate=8e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps = 30000,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end = True
  )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # get the data collator with our tokenizer.
early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

# Set the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = eng_tokenized['train'],
    eval_dataset = eng_tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
    callbacks = [early_stopping]
)

# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1500,0.328500,1.582880,0.634400
3000,0.390000,1.238372,0.639400
4500,0.402000,1.353696,0.627800
6000,0.453900,0.926526,0.632200
7500,0.727700,0.870842,0.640200
9000,0.718000,0.855909,0.645200
10500,0.733600,0.891537,0.643200
12000,0.737600,0.860115,0.636200
13500,0.729900,0.858592,0.650600


TrainOutput(global_step=13500, training_loss=0.5801252735279224, metrics={'train_runtime': 3162.8567, 'train_samples_per_second': 75.881, 'train_steps_per_second': 9.485, 'total_flos': 6879182037882528.0, 'train_loss': 0.5801252735279224, 'epoch': 0.54})

In [ ]:
#evaluation
eval_results = trainer.evaluate(eng_tokenized["validation"])

print(eval_results)

{'eval_loss': 0.8131952881813049, 'eval_accuracy': 0.6524, 'eval_runtime': 39.7428, 'eval_samples_per_second': 125.809, 'eval_steps_per_second': 15.726, 'epoch': 1.14}


# Hyperparam optimization

This happens over a couple of days so you won't see all the results...

In [ ]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 4),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        save_steps = 500,
        learning_rate = learning_rate,
        max_steps = 20000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs,
        weight_decay=0.01,
    )

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # get the data collator with our tokenizer.
    early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

    # train a model
    trainer = transformers.Trainer(
      model = model,
      args = trainer_args,
      train_dataset = eng_tokenized['train'],
      eval_dataset = eng_tokenized['test'],
      tokenizer = tokenizer,
      data_collator = data_collator,
      compute_metrics = compute_accuracy,
      callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2023-11-04 18:01:38,558] A new study created in memory with name: no-name-835858eb-df13-4840-8db2-ad5f39e2e0cb


Step,Training Loss,Validation Loss,Accuracy
500,0.784000,0.907039,0.607200
1000,0.737600,0.856668,0.646800
1500,0.731800,0.845865,0.646400
2000,0.685300,0.853943,0.651200
2500,0.708400,0.870679,0.648600
3000,0.661400,0.915587,0.650000


[I 2023-11-04 18:16:59,001] Trial 0 finished with value: 0.6464 and parameters: {'learning_rate': 8.614481030877561e-06, 'batch_size': 8, 'num_train_epochs': 2}. Best is trial 0 with value: 0.6464.


Step,Training Loss,Validation Loss,Accuracy
500,0.516600,1.200757,0.603200
1000,0.447100,1.159511,0.636400
1500,0.449400,1.122921,0.639000
2000,0.724300,0.870877,0.636800
2500,0.729100,0.873301,0.646200
3000,0.681800,0.925987,0.646200
3500,0.677200,0.945620,0.646400


[I 2023-11-04 18:34:25,704] Trial 1 finished with value: 0.6368 and parameters: {'learning_rate': 1.1454408190285528e-05, 'batch_size': 8, 'num_train_epochs': 4}. Best is trial 0 with value: 0.6464.


Step,Training Loss,Validation Loss,Accuracy
500,0.339000,1.562798,0.625400
1000,0.350700,1.656677,0.631800
1500,0.407000,1.554337,0.638000
2000,0.463300,1.132292,0.626600
2500,0.799300,0.914358,0.623400
3000,0.741300,0.933985,0.633000
3500,0.737800,0.934976,0.635200
4000,0.780400,0.903414,0.629800
4500,0.760400,0.878136,0.627200
5000,0.767200,0.936191,0.632000


[W 2023-11-04 19:02:28,939] Trial 2 failed with parameters: {'learning_rate': 1.9448091400021653e-05, 'batch_size': 8, 'num_train_epochs': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-16-23ee9406474e>", line 41, in objective
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1838, in _inner_training_loop
    for step, inputs in enumerate(epoch_iterator):
  File "/usr/local/lib/python3.10/dist-packages/accelerate/data_loader.py", line 460, in __iter__
    current_batch = send_to_device(current_batch, self.device)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/utils/operations.py", line 160, in send_to_dev

KeyboardInterrupt: ignored